# A/B тест маркетинговоой компании

Маркетинговые компании хотят проводить успешные кампании, но рынок сложен, и могут подойти несколько вариантов. Поэтому обычно они настраивают A/B-тесты, то есть рандомизированный процесс экспериментирования, в котором две или более версии переменной (веб-страница, элемент страницы, баннер и т. д.) одновременно показываются разным сегментам людей, чтобы определить, какая версия оказывает максимальное влияние и улучшает бизнес-показатели.
  
Компании заинтересованы в ответе на два вопроса:
- Будет ли кампания успешной?
- Если кампания окажется успешной, какая часть этого успеха может быть связана с рекламой?

В данном эксперименте посетители сайта были разделены на 2 группы:   
**the experimental group** - видели обычную рекламу,  
**the control group** - видели социальную рекламу или ничего в том месте, где обычно находится реклама.  

**Цель данного исследования:**  
    роанализировать группы, выяснить, была ли реклама успешной, сколько компания может заработать на ней и является ли разница между группами статистически значимой.

## Загрузка данных и подготовка к анализу

### Загрузка библиотек

In [1]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import numpy as np
import warnings

Уберем ограничение по выводу строк, колонок и символов в записи и включаем игнорирование ошибок Jupiter.

In [2]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

#Игнорируем предупреждения Jupiter
warnings.filterwarnings('ignore')

Прописываем темы визуализации.

In [3]:
sns.set_style('darkgrid')
plt.style.use('ggplot')

### Загрузка данных

Команда для загрузки датасета из Kaggle.

In [4]:
#kaggle datasets download -d faviovaz/marketing-ab-testing

Описание датасета

In [5]:
#https://www.kaggle.com/datasets/faviovaz/marketing-ab-testing

Запишем датафреймм marketing_AB.csv в датасет df.

In [6]:
df = pd.read_csv('downloads/marketing_AB.csv', index_col=0)

In [7]:
print(df.info())
display(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user id        588101 non-null  int64 
 1   test group     588101 non-null  object
 2   converted      588101 non-null  bool  
 3   total ads      588101 non-null  int64 
 4   most ads day   588101 non-null  object
 5   most ads hour  588101 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 27.5+ MB
None


,user id,test group,converted,total ads,most ads day,most ads hour
0,1069124,ad,False,130,Monday,20
1,1119715,ad,False,93,Tuesday,22
2,1144181,ad,False,21,Tuesday,18
3,1435133,ad,False,355,Tuesday,10
4,1015700,ad,False,276,Friday,14


В нашем распоряжении датафрейм из 6 колонок и 588100 строк.  
- `user id` - id пользователя (уникальные),
- `test group` - группа тестирования: "ad" если пользователь видел рекламу, "psa" пользователь видел социальное объявление,
- `converted`- True - если пользователь купил продукт, False - если не купил,
- `total ads` - количество рекламы, которую видел пользователь,
- `most ads day` - день, когда пользователь видел наибольшее количество рекламы,
- `most ads hour` - время (час), когда пользователь видел наибольшее количество рекламы.


Сохраним данные о размере датафрейма в перем енную start_shape.

In [8]:
df.memory_usage(deep=True).mean() / 1024 ** 2

12.51066289629255

In [9]:
start_shape = df.shape

### Предобработка данных

#### Переименование столбцов

Приведем название столбцов к "змеиному регистру".

In [10]:
df.columns = [re.sub(' ', '_',  i) for i in df.columns]
print(df.columns)

Index(['user_id', 'test_group', 'converted', 'total_ads', 'most_ads_day',
       'most_ads_hour'],
      dtype='object')


Переименование прошло успешно.

#### Проверка на пропуски

Проверим датафрейм на пропуски.

In [11]:
print(f'Пропусков в датафреме \n{df.isna().sum()}')

Пропусков в датафреме 
user_id          0
test_group       0
converted        0
total_ads        0
most_ads_day     0
most_ads_hour    0
dtype: int64


Пропусков нет.

#### Проверка на дубли

Проверим датафрем на явные дубли.

In [12]:
print(f'Явных дублей в датафреме {df.duplicated().sum()}')

Явных дублей в датафреме 0


Явных дублей нет.  
Проверим датафрейм на неявные дубли.  
Посмотрим, не встречается ли один пользователь дважды в `user_id`.

In [13]:
print(f'Дублей в user_id - {df.user_id.duplicated().sum()}')

Дублей в user_id - 0


Повторов нет.

Посмотрим, нет ли неявных дублей в тестовых группах.

In [14]:
print('Группы в колонке test_group:', df['test_group'].unique())

Группы в колонке test_group: ['ad' 'psa']


И нет ли неявных дублей в колонке `most_ads_day`

In [15]:
print('Значения в колонке most_ads_day:', df['most_ads_day'].unique())

Значения в колонке most_ads_day: ['Monday' 'Tuesday' 'Friday' 'Saturday' 'Wednesday' 'Sunday' 'Thursday']


Явных и неявных дублей нет.

#### Изменение формата даных

Посмотрим еще раз, какого формата у нас данные в колонках, а также сколько памяти занимает наш датафрейм.

In [16]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        588101 non-null  int64 
 1   test_group     588101 non-null  object
 2   converted      588101 non-null  bool  
 3   total_ads      588101 non-null  int64 
 4   most_ads_day   588101 non-null  object
 5   most_ads_hour  588101 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 87.6 MB


Мы видим, что датафрейм занимает 87,6 MB в памяти, это немного, но при этом можно улучшить этот показатель, т.к. у нас есть целочисленные значения в формате int6464, а также строчные значения там, где по сути категории.

Сначала применим формат категорий к `test_group` (всего 2 значения) и `most_ads_day` (7 значений).

In [17]:
for i in ['test_group', 'most_ads_day']:
    df[i] = df[i].astype('category')

Также поменяем формат в целочисленных колонках на минимально возможный.

In [18]:
for i in ['most_ads_hour', 'total_ads', 'user_id']:
    print('Колонка', i, 'значения от', df[i].min(), 'до', df[i].max())

Колонка most_ads_hour значения от 0 до 23
Колонка total_ads значения от 1 до 2065
Колонка user_id значения от 900000 до 1654483


Мы знаем, что для целочисленных значений бывают основные параметры:
**uint8**:  
min = 0  
max = 255   
**int8**:  
min = -128  
max = 127   
**int16**:  
min = -32768  
max = 32767    
**int32**:  
min = −2147483647  
max = 2147483647
А это значит, что колонку `most_ads_hour` мы можем привести к формату **uint8**, `total_ads` к **int16**, а `user_id` к **int32**

In [19]:
df['most_ads_hour'] = df['most_ads_hour'].astype('uint8')
df['total_ads'] = df['total_ads'].astype('int16')
df['user_id'] = df['user_id'].astype('int32')

И посмотрим, какого формата сейчас данные в колонках и сколько памяти занимает датафрейм.

In [20]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 588101 entries, 0 to 588100
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   user_id        588101 non-null  int32   
 1   test_group     588101 non-null  category
 2   converted      588101 non-null  bool    
 3   total_ads      588101 non-null  int16   
 4   most_ads_day   588101 non-null  category
 5   most_ads_hour  588101 non-null  uint8   
dtypes: bool(1), category(2), int16(1), int32(1), uint8(1)
memory usage: 10.1 MB


Мы видим, что формат данных изменен и датафрейм теперь занимает на 88% памяти меньше.

**Вывод**  
Мы подготовили данные к анализу:
- переименовали столбцы и привели их к “змеиному регистру”,
- проверили на пропуски датафрейм, 
- проверили на явные и неявные дубли,
- изменили форматы данных в колонках для более быстрой обработки.